In [ ]:
from typing import Optional, Any
from pydantic import BaseModel
from fastapi import FastAPI
from fastapi import HTTPException
from fastapi import status

from fastapi import Response
from fastapi import Header
from fastapi import Query
from fastapi import Path
from fastapi import Depends

from time import sleep
import nest_asyncio
import uvicorn

In [ ]:
import sys
sys.path.append("fastapi_files/")

In [ ]:
from fastapi_files.models import Curso

In [ ]:
def fake_db():
    try:
        print('Abrindo conexao com o banco de dados...')
        sleep(2)
    finally:
        print('Fechando conexao com o banco de dados')
        sleep(2)

In [ ]:
app = FastAPI()
cursos = {
    1: {
        "titulo": "adifuadfi",
        "aulas": '2323',
        "horas": '21:64'
    },
    2: {
        "titulo": "logica",
        "aulas": '222',
        "horas": '22:12'
    }
}

In [ ]:
@app.get('/cursos')
async def get_cursos(db: Any = Depends(fake_db)):
    return cursos

@app.get('/cursos/{curso_id}')
async def get_curso(curso_id: int = Path(title='ID do curso', description='Deve ser entre 1 e 2', gt=0, lt=3), db: Any = Depends(fake_db)):
    try:
        curso = cursos[curso_id]
        return curso
    except KeyError:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND,
                            detail='Curso nao encontrado')


In [ ]:
#passando um status depois de criado com o 201
@app.post('/cursos', status_code=status.HTTP_201_CREATED)
async def post_curso(curso: Curso, db: Any = Depends(fake_db)):
    if curso.id not in cursos:
        next_id: int = len(cursos) + 1
        curso.id = next_id
        cursos[next_id] = curso
        return curso
    else:
        raise HTTPException(status_code=status.HTTP_409_CONFLICT, defail=f"Ja existe um curso com o ID {curso.id }")

In [ ]:
@app.put('/cursos/{curso_id}')
async def put_curso(curso_id: int, curso: Curso, db: Any = Depends(fake_db)):
    if curso_id in cursos:
        curso.id = curso_id
        cursos[curso_id] = curso
        return curso
    else:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail=f'Não foi encontrado o id {curso_id}')

In [ ]:
#uma função sempre retorna algo, mesmo que None, sempre retorna algum valor, retorno de um Json ou Response
@app.delete('/cursos/{curso_id}')
async def delete_curso(curso_id: int, db: Any = Depends(fake_db)):
    if curso_id in cursos:
        del cursos[curso_id]
        #return JSONResponse(status_code=status.HTTP_204_NO_CONTENT)
        return Response(status_code=status.HTTP_204_NO_CONTENT)
    else:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail=f'Não foi encontrado o id {curso_id}')

In [ ]:
@app.get('/calculadora')
async def calcular(a: int=Query(gt=5),b: int=Query(gt=4),c: int=Query(gt=4), x_geek:str = Header(print='isssssssssd'), db: Any = Depends(fake_db)):
    soma = a + b + c
    print(f'X-geek: {x_geek}')
    return {"resultado": soma}

In [ ]:
if __name__ == '__main__':
    nest_asyncio.apply()
    uvicorn.run(app, host="0.0.0.0", port=8000)